# Generate Credit Card Transactions
**This notebook generates credit card transactions and randomly injects fraud chain attacks.**


---

## Contents

1. [Background](#Background)
1. [Setup](#Setup)
1. [Generate Transactions](#Generate-Transactions)
1. [Inject Fradulent Transactions](#Inject-Fradulent-Transactions)
1. [Save Generated Data](#Save-Generated-Data)

### Background
This notebook generates random credit card transactions for 10K users over a period of 5 months. In an ideal scenario, these historical transactions would be accumulated into a data lake/store for batch processing so as to derive insights and analytics about this data. Credit card numbers can be bought in bulk on the dark web through previous leaks or hacks of organizations that store this sensitive data. Fraudsters will buy these card lists and attempt to make as many transactions as possible with the stolen numbers until the card is blocked. These fraud chain attacks typically happen in a short time frame and can be easily spotted amongst historical transactions. This is because the velocity of transactions during the attack significantly differs from that of cardholder’s usual spending pattern. This notebook is optional to run. The generated data already exists in the `./data` folder for you to use. Re-run this notebook if you desire to re-populate fresh data or understand the whole process of how this dataset was generated.

### Setup

#### Prerequisites 

In [1]:
!pip install Faker

distutils: /home/ec2-user/anaconda3/envs/python3/include/python3.6m/UNKNOWN
sysconfig: /home/ec2-user/anaconda3/envs/python3/include/python3.6m
user = False
home = None
root = None
prefix = None
distutils: /home/ec2-user/anaconda3/envs/python3/include/python3.6m/UNKNOWN
sysconfig: /home/ec2-user/anaconda3/envs/python3/include/python3.6m
user = False
home = None
root = None
prefix = None


#### Imports 

In [2]:
from botocore.client import ClientError
from collections import defaultdict
from faker import Faker
import pandas as pd
import numpy as np
import sagemaker
import datetime
import hashlib
import random
import boto3
import math
import os

#### Seed for Reproducibility

In [3]:
faker = Faker()
faker.seed_locale('en_US', 0)

In [4]:
SEED = 123
random.seed(SEED)
np.random.seed(SEED)
faker.seed_instance(SEED)

#### Constants 

In [5]:
TOTAL_UNIQUE_TRANSACTIONS = 500 * 1000 
TOTAL_UNIQUE_USERS = 10000 
TOTAL_UNIQUE_CONSUMERS = 10000 
BUCKET = sagemaker.Session().default_bucket()

### Generate Transactions

#### Generate Unique Credit Card Numbers 
<p> Credit card numbers are uniquely assigned to users. Since, there are 10K users, we would want to generate 10K unique card numbers.</p>

In [6]:
def generate_unique_credit_card_numbers(n: int) -> list:
    cc_ids = set()
    for _ in range(n):
        cc_id = faker.credit_card_number(card_type='visa')
        cc_ids.add(cc_id)
    return list(cc_ids) 

In [7]:
credit_card_numbers = generate_unique_credit_card_numbers(TOTAL_UNIQUE_USERS)

In [8]:
assert len(credit_card_numbers) == TOTAL_UNIQUE_USERS 
assert len(credit_card_numbers[0]) == 16 # validate if generated number is 16-digit

In [9]:
# inspect random sample of credit card numbers 
random.sample(credit_card_numbers, 5)

['4061703683147422',
 '4827863299328132',
 '4298491674363043',
 '4926621427190695',
 '4782265458641458']

#### Generate Consumer IDs
Since we are training a consumer fraud detection model, we need a set of unique consumer IDs that we can randomly associate with each credit card transaction. The relationship between Consumer IDs to Credit Cards is one-to-many; a single Consumer can be tied to one or more credit instruments.

In [10]:
# We use Faker's Basic Bank Account Number (bban) module as a proxy for Consumer ID
def generate_unique_consumer_ids(n: int) -> list:
    id_list = set()
    for _ in range(n):
        bban_id = faker.unique.bban()
        id_list.add(bban_id)
    return list(id_list) 

In [11]:
consumer_ids = generate_unique_consumer_ids(TOTAL_UNIQUE_CONSUMERS)

In [12]:
assert len(consumer_ids) == TOTAL_UNIQUE_CONSUMERS

In [13]:
random.sample(consumer_ids, 5)

['ZKLI02793510270940',
 'XVIY60629876242488',
 'GAUY17387967775851',
 'NXZH89543483036593',
 'PVHV40060020147730']

#### Generate Time Series
<p>Generate 5.4 Million random timestamps spread across a period of 5 months (2020-01-01 to 2020-06-01) in sorted order.</p>
<b>Note:</b> The timestamps are NOT unique themselves. We can have 2 or more transactions occurring at the same time coming from different users. 

In [14]:
def generate_timestamps(n: int) -> list:
    # Use timeframe of single month, Jan 2021
    start = datetime.datetime.strptime('2021-01-01 00:00:00', '%Y-%m-%d %H:%M:%S')
    end = datetime.datetime.strptime('2021-01-31 23:59:59', '%Y-%m-%d %H:%M:%S')
    timestamps = list()
    for _ in range(n):
        timestamp = faker.date_time_between(start_date=start, end_date=end, tzinfo=None).strftime('%Y-%m-%d %H:%M:%S')
        timestamps.append(timestamp)
    timestamps = sorted(timestamps)
    return timestamps

In [15]:
timestamps = generate_timestamps(TOTAL_UNIQUE_TRANSACTIONS)

In [16]:
assert len(timestamps) == TOTAL_UNIQUE_TRANSACTIONS

In [17]:
# inspect random sample of timestamps
random.sample(timestamps, 5)

['2021-01-11 18:57:17',
 '2021-01-12 01:43:32',
 '2021-01-28 16:19:30',
 '2021-01-02 15:51:50',
 '2021-01-06 04:10:47']

#### Generate Random Transaction Amounts 
<p>The transaction amounts are presumed to follow Pareto distribution, as it is logical for consumers to make many more smaller purchases than large ones. The break down of the distribution is shown in the table below.</p>


| Percentage        | Range (Amount in $)     |
| :-------------: | :----------: |
|  5\% | 0.01 to 1    |
| 7.5\%   | 1 to 10 |
| 52.5\%   | 10 to 100 |
| 25\%   | 100 to 1000 |
| 10\%   | 1000 to 10000 |

In [18]:
def get_random_transaction_amount(start: float, end: float) -> float:
    amt = round(np.random.uniform(start, end), 2)
    return amt

In [19]:
distribution_percentages = {0.05: (0.01, 1.01), 
                            0.075: (1, 11.01),
                            0.525: (10, 100.01),
                            0.25: (100, 1000.01),
                            0.10: (1000, 10000.01)}

In [20]:
amounts = []

for percentage, span in distribution_percentages.items():
    n = int(TOTAL_UNIQUE_TRANSACTIONS * percentage)
    start, end = span
    for _ in range(n):
        amounts.append(get_random_transaction_amount(start, end+1))
        
random.shuffle(amounts)

In [21]:
assert len(amounts) == TOTAL_UNIQUE_TRANSACTIONS

In [22]:
# inspect random sample of transaction amounts
random.sample(amounts, 5)

[172.52, 23.05, 4109.33, 81.44, 93.3]

#### Generate Credit Card Transactions
<br>
<div style="text-align: justify">
Using the random credit card numbers, timestamps and transaction amounts generated in the above steps, 
we can generate random credit card transactions by combining them. The transaction id for the transaction is the md5
hash of the above mentioned entities.
</div>

In [23]:
def generate_transaction_id(timestamp: str, credit_card_number: str, transaction_amount: float) -> str:
    hashable = f'{timestamp}{credit_card_number}{transaction_amount}'
    hexdigest = hashlib.md5(hashable.encode('utf-8')).hexdigest()
    return hexdigest

In [24]:
transactions = []
for timestamp, amount in zip(timestamps, amounts):
    credit_card_number = random.choice(credit_card_numbers)
    consumer_id = random.choice(consumer_ids)
    transaction_id = generate_transaction_id(timestamp, credit_card_number, amount)
    transactions.append({'tid': transaction_id, 
                         'event_time': timestamp, 
                         'cc_num': credit_card_number, 
                         'consumer_id': consumer_id,
                         'amount': amount, 
                         'fraud_label': 0})

In [25]:
assert len(transactions) == TOTAL_UNIQUE_TRANSACTIONS

In [26]:
# inspect random sample of credit card transactions
random.sample(transactions, 1)

[{'tid': '812536f7c2a60fa3249f304de840ec7f',
  'event_time': '2021-01-13 19:19:07',
  'cc_num': '4915709307398000',
  'consumer_id': 'OECD47265252405706',
  'amount': 48.54,
  'fraud_label': 0}]

### Inject Fradulent Transactions
<p> A typical fraud chain looks like the one as shown in the image below.</p>
<img src="./images/fraud_pattern.png" />

In [27]:
FRAUD_RATIO = 0.0025 # percentage of transactions that are fraudulent
NUMBER_OF_FRAUDULENT_TRANSACTIONS = int(FRAUD_RATIO * TOTAL_UNIQUE_TRANSACTIONS)
ATTACK_CHAIN_LENGTHS = [3, 4, 5, 6, 7, 8, 9, 10]

#### Create Transaction Chains 

In [28]:
visited = set()
chains = defaultdict(list)

In [29]:
def size(chains: dict) -> int:
    counts = {key: len(values)+1 for (key, values) in chains.items()}
    return sum(counts.values())

In [30]:
def create_attack_chain(i: int):
    chain_length = random.choice(ATTACK_CHAIN_LENGTHS)
    for j in range(1, chain_length):
        if i+j not in visited:
            if size(chains) == NUMBER_OF_FRAUDULENT_TRANSACTIONS:
                break
            chains[i].append(i+j)
            visited.add(i+j)

In [31]:
while size(chains) < NUMBER_OF_FRAUDULENT_TRANSACTIONS:
    i = random.choice(range(TOTAL_UNIQUE_TRANSACTIONS))
    if i not in visited:
        create_attack_chain(i)
        visited.add(i)

In [32]:
assert size(chains) == NUMBER_OF_FRAUDULENT_TRANSACTIONS

#### Modify Transactions with Fraud Chain Attacks 

In [33]:
def generate_timestamps_for_fraud_attacks(timestamp: str, chain_length: int) -> list:
    timestamps = []
    timestamp = datetime.datetime.strptime(timestamp, '%Y-%m-%d %H:%M:%S')
    for _ in range(chain_length):
        # interval in seconds between fraudulent attacks
        delta = random.randint(30, 120)
        current = timestamp + datetime.timedelta(seconds=delta)
        timestamps.append(current.strftime('%Y-%m-%d %H:%M:%S'))
        timestamp = current
    return timestamps 

In [34]:
def generate_amounts_for_fraud_attacks(chain_length: int) -> list:
    amounts = []
    for percentage, span in distribution_percentages.items():
        n = math.ceil(chain_length * percentage)
        start, end = span
        for _ in range(n):
            amounts.append(get_random_transaction_amount(start, end+1))
    return amounts[:chain_length]

In [35]:
for key, chain in chains.items():
    transaction = transactions[key]
    timestamp = transaction['event_time']
    cc_num = transaction['cc_num']
    consumer_id = transaction['consumer_id']
    amount = transaction['amount']
    transaction['fraud_label'] = 1
    inject_timestamps = generate_timestamps_for_fraud_attacks(timestamp, len(chain))
    inject_amounts = generate_amounts_for_fraud_attacks(len(chain))
    random.shuffle(inject_amounts)
    for i, idx in enumerate(chain):
            original_transaction = transactions[idx]
            inject_timestamp = inject_timestamps[i]
            original_transaction['event_time'] = inject_timestamp
            original_transaction['fraud_label'] = 1
            original_transaction['cc_num'] = cc_num
            original_transaction['consumer_id'] = consumer_id
            original_transaction['amount'] = inject_amounts[i]
            original_transaction['tid'] = generate_transaction_id(inject_timestamp, cc_num, amount)
            transactions[idx] = original_transaction

#### Transform Transactions to Pandas DataFrame

In [36]:
transactions_df = pd.DataFrame(transactions)

In [37]:
fraud_transactions = transactions_df[transactions_df.fraud_label.eq(1)]
fraud_transactions.head()

,tid,event_time,cc_num,consumer_id,amount,fraud_label
831,6ab2735d53cc56845db49b68d6fd4fa5,2021-01-01 01:15:53,4375860660126547,SVQZ42593714587980,80.73,1
832,757055fbb628f9ffbbd1534c0ff3c2cb,2021-01-01 01:17:47,4375860660126547,SVQZ42593714587980,96.31,1
833,62d0061e6b329a16b200805f785468db,2021-01-01 01:18:54,4375860660126547,SVQZ42593714587980,0.60,1
834,925284968f498fbc91b4d3bb6cfba18e,2021-01-01 01:20:01,4375860660126547,SVQZ42593714587980,39.81,1
835,ceaa98fb2067c0ef3bd1f48649602480,2021-01-01 01:21:16,4375860660126547,SVQZ42593714587980,385.13,1


In [38]:
assert fraud_transactions.count()[0] == NUMBER_OF_FRAUDULENT_TRANSACTIONS

### Save Generated Data
<p> The generated raw transactions data will be used by the next step = SageMaker PySpark Processing Job to do aggregations on the raw data columns and derive new features which are useful for model training in the later steps.
The generated data is saved locally and then copied to S3 bucket.</p>

#### Save Transactions Data to Local Folder ./data and upload to S3

In [39]:
data_dir = os.path.join(os.getcwd(), 'data/raw')
os.makedirs(data_dir, exist_ok=True)

transactions_df.to_csv(f'{data_dir}/transactions.csv', index=False)

In [40]:
BASE_PREFIX = "sagemaker-featurestore-blog"

INPUT_KEY_PREFIX = os.path.join(BASE_PREFIX, 'raw')
print (INPUT_KEY_PREFIX)

sagemaker-featurestore-blog/raw


In [41]:
transactions_df.to_csv(f's3://{BUCKET}/{INPUT_KEY_PREFIX}/transactions.csv', index=False)